In [4]:
from pathlib import Path
import json
from pycocotools.coco import COCO

In [17]:
dataset_root = Path('/home/kuntik/dev/datasets/dental_exported')
task_name = 'task_dental_rtg'

In [20]:
img_roots = [dataset_root / (task_name + str(i) + '/images') for i in range(1,7)]
ann_roots = [dataset_root / (task_name + str(i) + '/annotations') for i in range(1,7)]
img_names = []
duplicit_img_names = []
for root in img_roots:
    names = [x.name for x in root.iterdir()]
    for name in names:
        if name not in img_names:
            img_names.append(name)
        else:
            duplicit_img_names.append(name)
print(duplicit_img_names)
print(len(duplicit_img_names))
print(len(img_names))

['84.png', '97.png', '87.png', '92.png', '98.png', '94.png', '9.png', '89.png', '91.png', '85.png', '88.png', '90.png', '96.png', '83.png', '81.png', '86.png', '99.png', '80.png', '95.png', '93.png', '82.png', '97.png', '92.png', '98.png', '94.png', '91.png', '90.png', '96.png', '99.png', '95.png', '93.png']
31
2599


In [39]:
with open(ann_roots[0]/ 'instances_default.json', 'r') as f:
    data = json.load(f)

merged = {}
merged['categories'] = data['categories']
merged['images'] = []
merged['annotations'] = []

img_names = []

img_id = 0
ann_id = 0

for ann_file in ann_roots:
    with open(ann_file/ 'instances_default.json', 'r') as f:
        data = json.load(f)
    
    for img in data['images']:
        if img['file_name'] not in img_names:
            img_names.append(img['file_name'])
        else:
            continue
        
        old_id = img['id']
        img['id'] = img_id
        img_id += 1
        merged['images'].append(img)
        for ann in data['annotations']:
            #only caries
            if ann['category_id'] != 1:
                continue
            if ann['image_id'] == old_id:
                ann['id'] = ann_id
                ann['image_id'] = img['id']
                ann_id += 1
                merged['annotations'].append(ann)

len(merged['images'])
len(merged['annotations'])
        

4172

In [ ]:

ann_paths = ['/home/kuntik/dev/datasets/dental_rtg_merged2/' + f_name + '.json' for f_name in range(1,5)]
with open(ann_paths[0], 'r') as f:
    data = json.load(f)

merged = {}
merged['categories'] = data['categories']
merged['images'] = []
merged['annotations'] = []

img_names = []

img_id = 0
ann_id = 0

for ann_path in ann_paths:
    with open(ann_path, 'r') as f:
        data = json.load(f)
    
    for img in data['images']:
        if img['file_name'] not in img_names:
            img_names.append(img['file_name'])
        else:
            continue
        
        old_id = img['id']
        img['id'] = img_id
        img_id += 1
        merged['images'].append(img)
        for ann in data['annotations']:
            #only caries
            if ann['category_id'] != 1:
                continue
            if ann['image_id'] == old_id:
                ann['id'] = ann_id
                ann['image_id'] = img['id']
                ann_id += 1
                merged['annotations'].append(ann)

len(merged['images'])
len(merged['annotations'])

In [35]:
print(ann_id)

4172


In [36]:
merged_dataset_annotations_path = Path('/home/kuntik/dev/datasets/dental_merged/annotations.json')
with open(merged_dataset_annotations_path, 'w') as f:
    json.dump(merged, f)

In [25]:
import shutil

merged_dataset_images_path = Path('/home/kuntik/dev/datasets/dental_merged/images')
merged_dataset_images_path.mkdir(parents=True)

img_names = []

for img_root in img_roots:
    for img in img_root.iterdir():
        if img not in img_names:
            img_names.append(img)
        else:
            continue
        shutil.copy(str(img_root/img.name), str(merged_dataset_images_path / img.name))
